In [41]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time

In [42]:
def init_browser():
    executable_path = {"executable_path": 'chromedriver.exe'}
    return Browser("chrome", **executable_path, headless=False)

In [45]:
def scrape():
    # initialize browser
    browser = init_browser()
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)
    time.sleep(1)
    mars_news = {}
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    mars_news["news_title"] = soup.find("div", class_="content_title").next_element.get_text()
    mars_news["news_p"] = soup.find("div", class_="article_teaser_body").get_text()
    return mars_news

In [46]:
scrape()

{'news_p': "Researchers using NASA's Mars Reconnaissance Orbiter have found eight sites where thick deposits of ice beneath Mars' surface are exposed.",
 'news_title': 'Steep Slopes on Mars Reveal Structure of Buried Ice'}

In [47]:
mars = scrape()
print(mars["news_title"])
print(mars["news_p"])

Steep Slopes on Mars Reveal Structure of Buried Ice
Researchers using NASA's Mars Reconnaissance Orbiter have found eight sites where thick deposits of ice beneath Mars' surface are exposed.


In [48]:
def scrape_img():
    browser = init_browser()
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')

    image = soup.find('a', class_='button fancybox')
    featured_image_url = "https://www.jpl.nasa.gov" + image["data-fancybox-href"]
    
    return featured_image_url

featured_image_url = scrape_img()
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19108_ip.jpg


In [49]:
def weather_scrape():
    browser = init_browser()
    url = "https://twitter.com/marswxreport?lang=en"
    
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    
    weather = soup.find_all('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
    
    for i in weather:
        j = i.text.split()
        if j[0] != "Sol":
            continue
        else:
            mars_weather = i.text
            break

    
        
    return mars_weather

mars_weather = weather_scrape()
print(mars_weather)

Sol 1934 (Jan 14, 2018), Sunny, high -21C/-5F, low -79C/-110F, pressure at 7.66 hPa, daylight 05:45-17:29


In [50]:
def fact_scrape():
    url = "https://space-facts.com/mars/"
    
    mars_table = pd.read_html(url)
    
    return mars_table

table = fact_scrape()

mars_fact_df = table[0]
mars_fact_df.columns = ["Characteristic", "Measurement"]

mars_fact_df.head()

,Characteristic,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [51]:
browser = init_browser()
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
base_url = "https://astrogeology.usgs.gov"

browser.visit(url)
time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'lxml')

url_list = []
title_list =[]
jpg_url_list =[]
hemisphere_image_urls = []

results = soup.find('div', class_="collapsible results")
links = results.find_all('a', class_="item product-item")
titles= results.find_all('h3')

for link in links:
    url_list.append(base_url+link["href"])
    
for title in titles:
    title_list.append(title.text)
    
for item in url_list:
    browser.visit(item)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    
    downloads = soup.find('div', class_="downloads")
    jpg = downloads.find_all('a')
    jpg_url = jpg[0]["href"]
    jpg_url_list.append(jpg_url)
    

for i in range(0,len(title_list)):
    hemisphere_image_urls.append({"title": title_list[i], "img_url": jpg_url_list[i]})

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 10.0.16299 x86_64)


In [ ]:
scraped_data = {}

scraped_data["news"] = mars
scraped_data["feat_img"] = featured_image_url
scraped_data["weather"] = mars_weather
scraped_data["facts"] = table
scraped_data["hemispheres"] = hemisphere_image_urls

In [ ]:
scraped_data

In [ ]:
mars_table_df = mars_fact_df.set_index("Characteristic")
mars_table_df

In [ ]:
mars_table_df.to_dict('list')

In [ ]:
mars_table_test=[]
for i in range(0, len(mars_fact_df)):
    print(mars_fact_df.iloc[i]['Characteristic'], mars_fact_df.iloc[i]['Measurement'])
    mars_table_test.append({"des": mars_fact_df.iloc[i]['Characteristic'], "mes": mars_fact_df.iloc[i]['Measurement']})
    
mars_table_test